<h1 align='center'>Welcome to my Notebook</h1>

In [9]:
import numpy as np 
import pandas as pd 

import matplotlib.pyplot as plt
#import plotly.express as px
import seaborn as sns

from lightgbm import LGBMClassifier
from sklearn.ensemble import GradientBoostingClassifier

from sklearn.model_selection import StratifiedKFold,KFold
from collections import Counter
from imblearn.under_sampling import RandomUnderSampler
from sklearn.metrics import classification_report, roc_auc_score, confusion_matrix, \
                            accuracy_score, f1_score, average_precision_score, \
                            fbeta_score, precision_recall_curve, auc
from sklearn.metrics import matthews_corrcoef

from sklearn.model_selection import train_test_split

import optuna
from optuna.samplers import TPESampler
sns.set(style='white', context='notebook', palette='deep')
%config Completer.use_jedi = False

In [10]:
csl_df = pd.read_csv('../../data/csl/CSL_tytl_PI.csv', index_col=0)
X = csl_df.drop(['trans_loss', 'transfus_yes'], axis=1, inplace=False)
y = csl_df['trans_loss'].values

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, stratify=y,
                                                    random_state=7)

<h1 align='center'>Exploratory Data Analysis</h1>

In [11]:
X

,Accrete,AdmBishop,Admcervpos,Admconsistency,Admcontract,AdmDBP,Admefface,Admpresent,AdmSBP,Analgesia,...,ThreatenedPB,threatpb9,UnspecHBP,uscar,version9,new_age,new_BMI,new_high_Age,new_high_BMI,Sitenum
MomID,,,,,,,,,,,,,,,,,,,,,
41-00002,0,8,8,8,99.0,83,0.0,77,150.0,7,...,0,0,0,1,0,37,33,1,1,41
41-00003,0,6,3,2,2.9,77,70.0,1,105.0,8,...,0,0,0,0,0,36,22,1,1,41
41-00004,0,5,3,2,3.3,65,70.0,1,122.0,8,...,0,0,0,0,0,34,31,1,1,41
41-00005,0,11,2,3,4.0,77,80.0,77,138.0,8,...,0,0,0,0,0,31,21,1,1,41
41-00006,0,11,1,3,5.0,98,100.0,77,134.0,8,...,0,0,0,0,0,21,22,1,1,41
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
52-07247,0,8,8,8,77.0,74,70.0,1,124.4,5,...,0,0,0,0,0,19,38,0,1,52
52-07248,0,8,8,8,99.0,74,0.0,2,124.4,5,...,0,0,0,1,0,34,24,1,1,52
52-07249,0,8,8,8,66.0,74,0.0,77,124.4,9,...,0,0,0,1,0,28,23,1,1,52


In [12]:
csl_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 185413 entries, 41-00002 to 52-07251
Columns: 197 entries, transfus_yes to Sitenum
dtypes: float64(4), int64(193)
memory usage: 280.1+ MB


In [13]:
X.skew()

Accrete           40.650960
AdmBishop         -1.136977
Admcervpos        -0.455299
Admconsistency    -0.407357
Admcontract        0.231978
                    ...    
new_age            0.232310
new_BMI            2.086880
new_high_Age      -0.673167
new_high_BMI      -0.549007
Sitenum           -0.027123
Length: 195, dtype: float64

<h1 align='center'>Categorical Features Data Analysis</h1>

In [14]:
%precision %.3f
#np.set_printoptions(precision=3)
print(f'{np.bincount(y)[1]/len(y):.4f}')
print(f'{np.bincount(y_train)[1]/len(y_train):.4f}')

0.0558
0.0558


In [1]:
fig1, ax1 = plt.subplots()
ax1.pie(csl_df['trans_loss'].value_counts(), labels=['No','Yes'], autopct='%1.1f%%')
plt.show()

NameError: name 'plt' is not defined

In [16]:
#fig = px.pie(train,values=train['Response'].value_counts(),names=['Class 0','Class 1'],hole=0.6,labels={0:'Response = 0'},color_discrete_sequence=px.colors.sequential.Sunset)
#fig.show(showlegend=True)
#plt.show()

In [17]:
#fig = px.pie(train,values=train['Response'].value_counts(),names=['Class 0','Class 1'],hole=0.6,labels={0:'Response = 0'},color_discrete_sequence=px.colors.sequential.Sunset)
#fig.show(showlegend=True)

In [2]:
sns.countplot(csl_df['trans_loss'])
plt.show()

NameError: name 'sns' is not defined

We can see from the above visualisations that:

* The Data is highly imbalanced

In [3]:
plt.figure(figsize=(20,10))
sns.barplot(csl_df['new_age'],csl_df['trans_loss'])
plt.show()

NameError: name 'plt' is not defined

In [4]:
sns.boxplot(csl_df['new_age'])
plt.show()

NameError: name 'sns' is not defined

In [5]:
plt.figure(figsize=(20,10))
sns.barplot(csl_df['new_BMI'],csl_df['trans_loss'])
plt.show()

NameError: name 'plt' is not defined

In [6]:
sns.boxplot(csl_df['new_BMI'])
plt.show()

NameError: name 'sns' is not defined

<h1 align='center'>Modelling</h1>

In [23]:
#X = all_features.iloc[:len(train),:]
#X_test = all_features.iloc[len(train):,:]

#kf = StratifiedKFold(n_splits=12,shuffle=True,random_state=42)

We define resplit our train and test set, and set up 12 Fold Stratified Cross Validation. It is important to stratify as this ensures that during our training and validation, we split according to target distribution

In [24]:
#for train_index,val_index in kf.split(X,y):
#    X_train,X_val = X.iloc[train_index],X.iloc[val_index],
#    y_train,y_val = y.iloc[train_index],y.iloc[val_index],

In [25]:
# MWB
print(X_train.shape)
print(X.shape)
print(y.shape)

(129789, 195)
(185413, 195)
(185413,)


Here we define our validation set

<h1 align='center'>Modelling Using Undersampling</h1>

We will use `imblearn`'s RandomUnderSampler to undersample from the majority class so that they match

In [31]:
sampler = RandomUnderSampler(sampling_strategy=1.0, random_state=7)
X_rus, y_rus = sampler.fit_resample(X_train, y_train)

# Try with no undersampling
#X_rus, y_rus = X_train, y_train

In [32]:
print(X_rus.shape)
print(y_rus.shape)
print(np.bincount(y_rus))

(14482, 195)
(14482,)
[7241 7241]


<h1 align='center'>Basic Gradient Boosting</h1>

In [34]:
from sklearn.metrics import matthews_corrcoef
from sklearn.utils import class_weight
gb_rus = GradientBoostingClassifier()   # If not undersampling
weights = class_weight.compute_sample_weight('balanced', y=y_rus)
gb_rus.fit(X_rus, y_rus, sample_weight=weights)
print(gb_rus.get_params())
y_pred = gb_rus.predict(X_test)
probs = gb_rus.predict_proba(X_test)
prob1 = probs[:, 1]  # Only positives
precision, recall, pr_thresh = precision_recall_curve(y_test, prob1)
print(confusion_matrix(y_pred=y_pred, y_true=y_test))
print(classification_report(y_test, y_pred))
print(f'ROC AUC prob1: {roc_auc_score(y_test, prob1)}')
print(f'MCC: {matthews_corrcoef(y_test, y_pred)}')
print(f'F2: {fbeta_score(y_test, y_pred, beta=2.0, average=None)[1]}')
print(f'PR_AUC: {auc(recall, precision)}')

{'ccp_alpha': 0.0, 'criterion': 'friedman_mse', 'init': None, 'learning_rate': 0.1, 'loss': 'deviance', 'max_depth': 3, 'max_features': None, 'max_leaf_nodes': None, 'min_impurity_decrease': 0.0, 'min_impurity_split': None, 'min_samples_leaf': 1, 'min_samples_split': 2, 'min_weight_fraction_leaf': 0.0, 'n_estimators': 100, 'n_iter_no_change': None, 'presort': 'deprecated', 'random_state': None, 'subsample': 1.0, 'tol': 0.0001, 'validation_fraction': 0.1, 'verbose': 0, 'warm_start': False}
[[35054 17467]
 [  350  2753]]
              precision    recall  f1-score   support

           0       0.99      0.67      0.80     52521
           1       0.14      0.89      0.24      3103

    accuracy                           0.68     55624
   macro avg       0.56      0.78      0.52     55624
weighted avg       0.94      0.68      0.77     55624

ROC AUC prob1: 0.8355697697594842
MCC: 0.26463499992703937
F2: 0.42182520225545467
PR_AUC: 0.2140558200374818


In [28]:
from sklearn.metrics import matthews_corrcoef
from sklearn.utils import class_weight
gb_rus = GradientBoostingClassifier()   # If not undersampling
weights = class_weight.compute_sample_weight('balanced', y=y_rus)
gb_rus.fit(X_rus, y_rus, sample_weight=weights)
print(gb_rus.get_params())
y_pred = gb_rus.predict(X_test)
probs = gb_rus.predict_proba(X_test)
prob1 = probs[:, 1]  # Only positives
precision, recall, pr_thresh = precision_recall_curve(y_test, prob1)
print(confusion_matrix(y_pred=y_pred, y_true=y_test))
print(classification_report(y_test, y_pred))
print(f'ROC AUC prob1: {roc_auc_score(y_test, prob1)}')
print(f'MCC: {matthews_corrcoef(y_test, y_pred)}')
print(f'F2: {fbeta_score(y_test, y_pred, beta=2.0, average=None)[1]}')
print(f'PR_AUC: {auc(recall, precision)}')

[[35363 17158]
 [  374  2729]]
              precision    recall  f1-score   support

           0       0.99      0.67      0.80     52521
           1       0.14      0.88      0.24      3103

    accuracy                           0.68     55624
   macro avg       0.56      0.78      0.52     55624
weighted avg       0.94      0.68      0.77     55624

ROC AUC prob1: 0.8378781967869053
MCC: 0.2647088495812209
F2: 0.4224588996563361
PR_AUC: 0.22394914990846596


<h1 align='center'>Hyperparameter Tuning</h1>

In [37]:
def create_model(trial):
    n_estimators = trial.suggest_int('n_estimators', 10, 200)
    subsample = trial.suggest_float('subsample', 0.9, 1.0)
#    max_depth = trial.suggest_int('max_depth',3,20)
    learning_rate = trial.suggest_uniform('learning_rate',0.01,0.2)
    min_samples_leaf = trial.suggest_int('min_samples_leaf',1,50)
#    model = GradientBoostingClassifier(n_estimators=n_estimators, subsample=subsample, 
#                           max_depth=max_depth, learning_rate=learning_rate, 
#                           min_samples_leaf=min_samples_leaf)
    model = GradientBoostingClassifier(n_estimators=n_estimators, subsample=subsample, 
                           learning_rate=learning_rate, 
                           min_samples_leaf=min_samples_leaf)
    return model

def objective(trial):
    model = create_model(trial)
    model.fit(X_rus, y_rus)
    y_pred = model.predict(X_test)
    probs = model.predict_proba(X_test)
    prob1 = probs[:, 1]  # Only positives
    precision, recall, pr_thresh = precision_recall_curve(y_test, prob1)
    roc = roc_auc_score(y_test, prob1)
    mcc = matthews_corrcoef(y_test, y_pred)  # MWB
    f2 = fbeta_score(y_test, y_pred, beta=2.0, average=None)[1]
    pr_auc = auc(recall, precision)
    avg_precision = average_precision_score(y_test, prob1)
    print(f'ROC: {roc}')
    print(f'PR_AUC: {pr_auc}')
    print(f'avg_prec: {avg_precision}')
    print(f'MCC: {mcc}')
    return mcc
#    return roc

sampler = TPESampler(seed=42)
study = optuna.create_study(sampler=sampler,direction='maximize')
#study.optimize(objective,n_trials=60)
study.optimize(objective,n_trials=5)

[I 2021-12-03 16:31:43,052] A new study created in memory with name: no-name-f761ccaf-141e-4157-acbe-781672b411fb


{'ccp_alpha': 0.0, 'criterion': 'friedman_mse', 'init': None, 'learning_rate': 0.1, 'loss': 'deviance', 'max_depth': 3, 'max_features': None, 'max_leaf_nodes': None, 'min_impurity_decrease': 0.0, 'min_impurity_split': None, 'min_samples_leaf': 1, 'min_samples_split': 2, 'min_weight_fraction_leaf': 0.0, 'n_estimators': 100, 'n_iter_no_change': None, 'presort': 'deprecated', 'random_state': None, 'subsample': 1.0, 'tol': 0.0001, 'validation_fraction': 0.1, 'verbose': 0, 'warm_start': False}


[I 2021-12-03 16:31:48,134] Trial 0 finished with value: 0.26541888202503866 and parameters: {'n_estimators': 81, 'subsample': 0.9852142919229748, 'learning_rate': 0.14907884894416698, 'min_samples_leaf': 30}. Best is trial 0 with value: 0.26541888202503866.


ROC: 0.8365339805486274
PR_AUC: 0.2187978814719409
avg_prec: 0.2190332028833386
MCC: 0.26541888202503866
{'ccp_alpha': 0.0, 'criterion': 'friedman_mse', 'init': None, 'learning_rate': 0.1, 'loss': 'deviance', 'max_depth': 3, 'max_features': None, 'max_leaf_nodes': None, 'min_impurity_decrease': 0.0, 'min_impurity_split': None, 'min_samples_leaf': 1, 'min_samples_split': 2, 'min_weight_fraction_leaf': 0.0, 'n_estimators': 100, 'n_iter_no_change': None, 'presort': 'deprecated', 'random_state': None, 'subsample': 1.0, 'tol': 0.0001, 'validation_fraction': 0.1, 'verbose': 0, 'warm_start': False}


[I 2021-12-03 16:31:50,408] Trial 1 finished with value: 0.23982979998335768 and parameters: {'n_estimators': 39, 'subsample': 0.7467983561008608, 'learning_rate': 0.021035886311957897, 'min_samples_leaf': 44}. Best is trial 0 with value: 0.26541888202503866.


...

ROC: 0.8287811128176754
PR_AUC: 0.1981845524610231
avg_prec: 0.19528415745177166
MCC: 0.2606340931049227
{'ccp_alpha': 0.0, 'criterion': 'friedman_mse', 'init': None, 'learning_rate': 0.1, 'loss': 'deviance', 'max_depth': 3, 'max_features': None, 'max_leaf_nodes': None, 'min_impurity_decrease': 0.0, 'min_impurity_split': None, 'min_samples_leaf': 1, 'min_samples_split': 2, 'min_weight_fraction_leaf': 0.0, 'n_estimators': 100, 'n_iter_no_change': None, 'presort': 'deprecated', 'random_state': None, 'subsample': 1.0, 'tol': 0.0001, 'validation_fraction': 0.1, 'verbose': 0, 'warm_start': False}


[I 2021-12-03 16:32:07,038] Trial 3 finished with value: 0.26448822218278967 and parameters: {'n_estimators': 168, 'subsample': 0.7637017332034828, 'learning_rate': 0.04454674376934912, 'min_samples_leaf': 10}. Best is trial 0 with value: 0.26541888202503866.


ROC: 0.835988073042655
PR_AUC: 0.217290475020333
avg_prec: 0.21754601022697295
MCC: 0.26448822218278967
{'ccp_alpha': 0.0, 'criterion': 'friedman_mse', 'init': None, 'learning_rate': 0.1, 'loss': 'deviance', 'max_depth': 3, 'max_features': None, 'max_leaf_nodes': None, 'min_impurity_decrease': 0.0, 'min_impurity_split': None, 'min_samples_leaf': 1, 'min_samples_split': 2, 'min_weight_fraction_leaf': 0.0, 'n_estimators': 100, 'n_iter_no_change': None, 'presort': 'deprecated', 'random_state': None, 'subsample': 1.0, 'tol': 0.0001, 'validation_fraction': 0.1, 'verbose': 0, 'warm_start': False}


[I 2021-12-03 16:32:10,914] Trial 4 finished with value: 0.26479441103956564 and parameters: {'n_estimators': 68, 'subsample': 0.8574269294896714, 'learning_rate': 0.092069553542002, 'min_samples_leaf': 15}. Best is trial 0 with value: 0.26541888202503866.


ROC: 0.8352831112540635
PR_AUC: 0.21418274875639715
avg_prec: 0.21426100750154575
MCC: 0.26479441103956564


In [45]:
def create_model(trial):
    n_estimators = trial.suggest_int('n_estimators', 10, 200)
    subsample = trial.suggest_float('subsample', 0.9, 1.0)
#    max_depth = trial.suggest_int('max_depth',3,20)
    learning_rate = trial.suggest_uniform('learning_rate',0.01,0.2)
    min_samples_leaf = trial.suggest_int('min_samples_leaf',1,50)
#    model = GradientBoostingClassifier(n_estimators=n_estimators, subsample=subsample, 
#                           max_depth=max_depth, learning_rate=learning_rate, 
#                           min_samples_leaf=min_samples_leaf)
    model = GradientBoostingClassifier(n_estimators=n_estimators, subsample=subsample, 
                           learning_rate=learning_rate, 
                           min_samples_leaf=min_samples_leaf)
    return model

def objective(trial):
    model = create_model(trial)
    model.fit(X_rus, y_rus)
    y_pred = model.predict(X_test)
    probs = model.predict_proba(X_test)
    prob1 = probs[:, 1]  # Only positives
    precision, recall, pr_thresh = precision_recall_curve(y_test, prob1)
    roc = roc_auc_score(y_test, prob1)
    mcc = matthews_corrcoef(y_test, y_pred)  # MWB
    f2 = fbeta_score(y_test, y_pred, beta=2.0, average=None)[1]
    pr_auc = auc(recall, precision)
    avg_precision = average_precision_score(y_test, prob1)
    print(f'ROC: {roc}')
    print(f'PR_AUC: {pr_auc}')
    print(f'avg_prec: {avg_precision}')
    print(f'MCC: {mcc}')
    return mcc
#    return roc

sampler = TPESampler(seed=42)
study = optuna.create_study(sampler=sampler,direction='maximize')
study.optimize(objective,n_trials=60)
#study.optimize(objective,n_trials=5)

[I 2021-12-03 17:30:52,893] A new study created in memory with name: no-name-a8cbb3b2-ae64-4f53-b9fb-402ad54ccc0c
[I 2021-12-03 17:30:58,099] Trial 0 finished with value: 0.2654547963716877 and parameters: {'n_estimators': 81, 'subsample': 0.9950714306409916, 'learning_rate': 0.14907884894416698, 'min_samples_leaf': 30}. Best is trial 0 with value: 0.2654547963716877.


ROC: 0.8363947179288591
PR_AUC: 0.2195439696948936
avg_prec: 0.2197653647796915
MCC: 0.2654547963716877


[I 2021-12-03 17:31:00,927] Trial 1 finished with value: 0.23982979998335768 and parameters: {'n_estimators': 39, 'subsample': 0.9155994520336203, 'learning_rate': 0.021035886311957897, 'min_samples_leaf': 44}. Best is trial 0 with value: 0.2654547963716877.


ROC: 0.8220709537034442
PR_AUC: 0.1855844912958978
avg_prec: 0.17698409946770524
MCC: 0.23982979998335768


[I 2021-12-03 17:31:08,962] Trial 2 finished with value: 0.2607222534891927 and parameters: {'n_estimators': 124, 'subsample': 0.9708072577796045, 'learning_rate': 0.013911053916202464, 'min_samples_leaf': 49}. Best is trial 0 with value: 0.2654547963716877.


...

ROC: 0.8375857182869988
PR_AUC: 0.22010403739433895
avg_prec: 0.2203033720764178
MCC: 0.26555255854579674


[I 2021-12-03 17:40:13,324] Trial 58 finished with value: 0.266099831248213 and parameters: {'n_estimators': 193, 'subsample': 0.9552103938844836, 'learning_rate': 0.12070273541315826, 'min_samples_leaf': 9}. Best is trial 14 with value: 0.2672634282490243.


ROC: 0.8371439785579253
PR_AUC: 0.21887153396686462
avg_prec: 0.21908779416964208
MCC: 0.266099831248213


[I 2021-12-03 17:40:22,499] Trial 59 finished with value: 0.2640860301261427 and parameters: {'n_estimators': 152, 'subsample': 0.9298245189364603, 'learning_rate': 0.1499302812458187, 'min_samples_leaf': 15}. Best is trial 14 with value: 0.2672634282490243.


ROC: 0.8361214358999582
PR_AUC: 0.2170292201803484
avg_prec: 0.21723927054901637
MCC: 0.2640860301261427


In [41]:
gb_params = study.best_params
gb_params['random_state'] = 42
gb = GradientBoostingClassifier(**gb_params)
gb.fit(X_rus, y_rus)
y_pred = gb.predict(X_test)
probs = gb.predict_proba(X_test)
prob1 = probs[:, 1]  # Only positives
precision, recall, pr_thresh = precision_recall_curve(y_test, prob1)
print(confusion_matrix(y_pred=y_pred, y_true=y_test))
print(classification_report(y_test, y_pred))
print(f'ROC AUC prob1: {roc_auc_score(y_test, prob1)}')
print(f'MCC: {matthews_corrcoef(y_test, y_pred)}')
print(f'F2: {fbeta_score(y_test, y_pred, beta=2.0, average=None)[1]}')
print(f'PR_AUC: {auc(recall, precision)}')

[[35182 17339]
 [  355  2748]]
              precision    recall  f1-score   support

           0       0.99      0.67      0.80     52521
           1       0.14      0.89      0.24      3103

    accuracy                           0.68     55624
   macro avg       0.56      0.78      0.52     55624
weighted avg       0.94      0.68      0.77     55624

ROC AUC pred: 0.7777299865315448
ROC AUC prob1: 0.8366437320840735
MCC: 0.26540691190037247
F2: 0.42278223945352167
PR_AUC: 0.21889157239149593


In [44]:
gb_params

{'n_estimators': 81,
 'subsample': 0.985,
 'learning_rate': 0.149,
 'min_samples_leaf': 30,
 'random_state': 42}

<h1 align='center'>Thanks and make sure to learn!</h1>